In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("MainDataSet1D.csv").tail(1000)

In [177]:
df.columns


Index(['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_SuperTrend',
       'Day_Candle', 'Day_CandleBody', 'Day_5EMA', 'Day_15EMA', 'Day_20EMA',
       'Day_50EMA', 'Day_200EMA', 'Day_RSI', 'Day_Date', 'Day_SwingMin',
       'Day_SwingMax', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Pivot',
       'Candle', 'CandleBody', '5EMA', '15EMA', '20EMA', '50EMA', '200EMA',
       'RSI', 'SuperTrend', 'Small_Swing_Min', 'Small_Swing_Max'],
      dtype='object')

In [178]:
status = (df['Pivot'] - df['Pivot'].shift(5) >= 200).shift(-6)
df["Side"] = np.where(status, "CE", "None")

In [179]:
df[df["Side"] != "None"]

,Day_Open,Day_High,Day_Low,Day_Close,Day_SuperTrend,Day_Candle,Day_CandleBody,Day_5EMA,Day_15EMA,Day_20EMA,...,5EMA,15EMA,20EMA,50EMA,200EMA,RSI,SuperTrend,Small_Swing_Min,Small_Swing_Max,Side
8,17051.20,17324.70,16931.40,16983.2,-1,Red,393.30,17178.163044,17516.154645,17597.529314,...,17178.163044,17516.154645,17597.529314,17603.258175,16372.222537,20.984057,-1,16782.4,22794.7,CE
11,17424.90,17489.80,17180.80,17196.7,-1,Red,309.00,17232.348309,17430.279576,17508.908473,...,17232.348309,17430.279576,17508.908473,17563.924168,16398.318587,41.278637,-1,16782.4,22794.7,CE
21,17276.00,17298.20,16966.40,16985.2,-1,Red,331.80,17191.698221,17303.013961,17353.617656,...,17191.698221,17303.013961,17353.617656,17467.298183,16481.558502,30.024273,-1,16782.4,22794.7,CE
22,16824.20,16840.10,16410.20,16614.2,-1,Red,429.90,16999.198814,17216.912216,17283.196927,...,16999.198814,17216.912216,17283.196927,17433.843352,16482.878318,19.599917,-1,16410.2,22794.7,CE
23,16773.20,16936.40,16688.20,16770.8,-1,Red,248.20,16923.065876,17161.148189,17234.397220,...,16923.065876,17161.148189,17234.397220,17407.841652,16485.743210,31.624649,-1,16410.2,22794.7,CE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,22561.60,22588.80,22409.45,22442.7,1,Red,179.35,22505.820323,22455.911387,22429.344817,...,22505.820323,22455.911387,22429.344817,22238.022009,20955.902575,46.707719,1,21137.2,22794.7,CE
611,22489.75,22499.05,22232.05,22302.5,1,Red,267.00,22438.046882,22436.734963,22417.264358,...,22438.046882,22436.734963,22417.264358,22240.550557,20969.301554,36.988077,1,21137.2,22794.7,CE
612,22231.20,22368.65,22185.20,22302.5,1,Green,183.45,22392.864588,22419.955593,22406.334419,...,22392.864588,22419.955593,22406.334419,22242.979947,20982.567210,36.988077,1,21137.2,22794.7,CE
613,22224.80,22307.75,21932.40,21957.5,1,Red,375.35,22247.743059,22362.148644,22363.588284,...,22247.743059,22362.148644,22363.588284,22231.784655,20992.268034,21.289492,1,21137.2,22794.7,CE


In [180]:
import pandas as pd
from tabulate import tabulate
class Trader:
    def __init__(self,initial_balance=100000):
        self.balance = initial_balance
        self.trad_book = pd.DataFrame(columns=['IndexName', 'Side', 'Status', 'Quantity', 'BuyPrice',
                                               'BuyDatetime', 'SellPrice', 'SellDatetime','SLValue',
                                               'TargetValue', 'PnL Status','PnL'])
        

    def backtest(self, df):
        open_order = None
        side = None
        tg_order = None
        sl_order = None

        for index, row in df.iterrows():
            if row["Side"] != "None" and open_order is None:
                open_order = row["Close"]
                side = row["Side"]
                sl_order, tg_order = (open_order - 1000, open_order + 1000) if side == "CE" else (open_order + 100, open_order - 100)
                symbol = "Nifty50"
                quantity = 10
                new_log = pd.DataFrame({'IndexName': [symbol],'Side':[side],'Status':["Open"] ,'Quantity': [quantity], 'BuyPrice': [open_order],
                                'BuyDatetime': [row["Datetime"]], 'SellDatetime': [None], 'SellPrice': [None],
                                'PnL': [0],'SLValue': [sl_order], 'TargetValue': [tg_order], 'PnL Status': [None] })
                self.trad_book = pd.concat([self.trad_book, new_log], ignore_index=True)
            if open_order is not None:
                if (side == "CE" and (row["Close"] >= tg_order or row["Close"] <= sl_order)) or \
                   (side == "PE" and (row["Close"] <= tg_order or row["Close"] >= sl_order)):
                    pnl = row["Close"] - open_order if side == "CE" else open_order - row["Close"]
                    self.balance += pnl
                    
                    index = self.trad_book.shape[0] -1
                    self.trad_book.at[index, 'SellPrice'] = row["Close"]
                    self.trad_book.at[index, 'SellDatetime'] = row["Datetime"]
                    self.trad_book.at[index, 'Status'] = "Done"
                    self.trad_book.at[index, 'PnL'] = pnl
                    self.trad_book.at[index, 'PnL Status'] = "Profit" if  pnl >0  else "Loss"
                    self.trad_book.at[index, 'SellDatetime'] = row["Datetime"]
                    open_order = None

    
    def stats(self):
            df = self.trad_book
            total_trade = len(df.index)
            pnl = df.PnL.sum()
            winners = len(df[df.PnL > 0])
            losers = len(df[df.PnL <= 0])
            win_ratio = round((winners / total_trade) * 100, 2)
    
            # Calculate CE and PE trades
        
            ce_trades = f"{(len(df[(df['Side'] == 'CE') & (df['PnL'] > 0)]) / len(df[df['Side'] == 'CE']) * 100):.2f}%" if len(df[df['Side'] == 'CE']) != 0 else 0
            pe_trades = f"{(len(df[(df['Side'] == 'PE') & (df['PnL'] > 0)]) / len(df[df['Side'] == 'PE']) * 100):.2f}%" if len(df[df['Side'] == 'PE']) != 0 else 0

    
            # Calculate additional metrics
            capital = self.balance
            max_win = round(df[df.PnL > 0].PnL.max(), 2) if winners > 0 else 0
            max_profit_sum = round(df[df.PnL > 0].PnL.sum(), 2) if winners > 0 else 0
            max_loss = round(df[df.PnL <= 0].PnL.min(), 2) if losers > 0 else 0
            max_loss_sum = round(df[df.PnL <= 0].PnL.sum(), 2) if losers > 0 else 0
            total_profit = round(df.PnL.sum(), 2)
            total_profit_percentage = round((total_profit / self.balance) * 100, 2)
    
            # Prepare the data for tabular representation
            # parameters = ['Total Trades', 'Capital', 'Total Wins', 'Total Losses', 'Win Ratio',
            #               'Max Win', 'Max Win Score','Max Loss', 'Max Loss Score','Total Profit', ' Grow Profit %',
            #               'CE Trades', 'PE Trades', 'Stategy Name']


            parameters = ['Total Trades', 'Capital', 'Total Wins', 'Total Losses', 'Win Ratio','Total Profit', 'Grow Profit %',
                              'Max Win', 'Max Win Score','Max Loss', 'Max Loss Score','CE Trades', 'PE Trades', 'Stategy Name']


        
            data_points = [total_trade, capital, winners, losers, f"{win_ratio}%",total_profit,f"{total_profit_percentage}%",
                           max_win, max_profit_sum,max_loss, max_loss_sum,ce_trades, pe_trades, "Test"]

        
            data = list(zip(parameters, data_points))
    
            # Print the tabular representation
            print(tabulate(data, headers=['Parameters', 'Values'], tablefmt='psql'))
    
            excel_file = "statistics.xlsx"
            try:
                existing_df = pd.read_excel(excel_file)
                new_data = pd.DataFrame([data_points], columns=parameters)
                updated_df = pd.concat([existing_df, new_data], ignore_index=True)
                updated_df.to_excel(excel_file, index=False)
            except FileNotFoundError:
                new_data = pd.DataFrame([data_points], columns=parameters)
                new_data.to_excel(excel_file, index=False)

trader = Trader()
trader.backtest(df)
trader.stats()


+----------------+--------------------+
| Parameters     | Values             |
|----------------+--------------------|
| Total Trades   | 92                 |
| Capital        | 103446.90000000002 |
| Total Wins     | 58                 |
| Total Losses   | 34                 |
| Win Ratio      | 63.04%             |
| Total Profit   | 3446.9             |
| Grow Profit %  | 3.33%              |
| Max Win        | 451.3              |
| Max Win Score  | 11157.75           |
| Max Loss       | -532.0             |
| Max Loss Score | -7710.85           |
| CE Trades      | 63.04%             |
| PE Trades      | 0                  |
| Stategy Name   | Test               |
+----------------+--------------------+


In [181]:
trader.trad_book

,IndexName,Side,Status,Quantity,BuyPrice,BuyDatetime,SellPrice,SellDatetime,SLValue,TargetValue,PnL Status,PnL
0,Nifty50,CE,Done,10,16983.20,2021-11-30 05:30:00,17166.9,2021-12-01 05:30:00,16883.20,17083.20,Profit,183.7
1,Nifty50,CE,Done,10,17196.70,2021-12-03 05:30:00,16912.2,2021-12-06 05:30:00,17096.70,17296.70,Loss,-284.5
2,Nifty50,CE,Done,10,16985.20,2021-12-17 05:30:00,16614.2,2021-12-20 05:30:00,16885.20,17085.20,Loss,-371.0
3,Nifty50,CE,Done,10,16770.80,2021-12-21 05:30:00,16955.4,2021-12-22 05:30:00,16670.80,16870.80,Profit,184.6
4,Nifty50,CE,Done,10,17072.60,2021-12-23 05:30:00,17233.2,2021-12-28 05:30:00,16972.60,17172.60,Profit,160.6
...,...,...,...,...,...,...,...,...,...,...,...,...
87,Nifty50,CE,Done,10,21995.85,2024-04-18 05:30:00,22147.0,2024-04-19 05:30:00,21895.85,22095.85,Profit,151.15
88,Nifty50,CE,Done,10,22336.40,2024-04-22 05:30:00,22570.35,2024-04-25 05:30:00,22236.40,22436.40,Profit,233.95
89,Nifty50,CE,Done,10,22475.85,2024-05-03 05:30:00,22302.5,2024-05-07 05:30:00,22375.85,22575.85,Loss,-173.35
90,Nifty50,CE,Done,10,22302.50,2024-05-08 05:30:00,21957.5,2024-05-09 05:30:00,22202.50,22402.50,Loss,-345.0
